# Algorithmic Market Efficiency - Data Preparation Code

In [ ]:
import numpy as np
import pandas as pd

import pickle
import warnings
import os
import sys

from sklearn.preprocessing import scale

## Firm-level factors

In [ ]:
# Firm-level factors from Gu et al. (2019)

df_factors = pd.read_csv('data/factors.csv')

df_factors.columns = map(str.lower, df_factors.columns)

df_factors = df_factors.drop(columns='sic2')

print(df_factors.columns)

## CRSP

In [ ]:
# CRSP database (retrieved Oct. 2019) : dropping of duplicates, computation of market capitalization

df_crsp = pd.read_csv('data/crsp.csv')

df_crsp = df_crsp.drop_duplicates(subset=['date', 'permno'])

df_crsp['shrout'] = df_crsp['shrout'] * 1000 #shrout is in 1000s
df_crsp['mktcap'] = df_crsp['shrout'] * abs(df_crsp['prc'])

df_crsp = df_crsp[['permno', 'date', 'ret', 'mktcap', 'shrout']]

print(df_crsp.columns)

In [ ]:
# First merge that forms df, with df_factors as a basis

df = pd.merge(df_factors, df_crsp, on=['permno', 'date'], how='left', validate='one_to_one')

df['yyyymm'] = (df['date'] / 100).astype(int)

df_factors = None
df_crsp = None

## Fama-French and Carhat factors

In [ ]:
# Fama-French aggregate factors, retrieved in Oct. 2019

df_ff = pd.read_csv('data/ff.csv', header=2, skipfooter=96, engine='python')

df_ff = df_ff.rename(columns={"Unnamed: 0": "yyyymm", "Mkt-RF": "mkt_rf"})
df_ff.columns = map(str.lower, df_ff.columns)

df_ff['rf'] = df_ff['rf'] / 100
df_ff['mkt_rf'] = df_ff['mkt_rf'] / 100
df_ff['smb'] = df_ff['smb'] / 100
df_ff['hml'] = df_ff['hml'] / 100

# Fama-French 5-factor variables, retrieved in May 2020

df_ff5 = pd.read_csv('data/ff5.csv', header=2, skipfooter=60, engine='python')

df_ff5 = df_ff5.rename(columns={"Unnamed: 0": "yyyymm", "Mkt-RF": "mkt_rf"})
df_ff5.columns = map(str.lower, df_ff5.columns)

df_ff5['rmw'] = df_ff5['rmw'] / 100
df_ff5['cma'] = df_ff5['cma'] / 100

# Momentum factor, retrieved in May 2020

df_mom = pd.read_csv('data/mom.csv', header=11, skipfooter=98, engine='python')

df_mom = df_mom.rename(columns={"Unnamed: 0": "yyyymm", "Mom   ": "mom"})

df_mom['mom'] = df_mom['mom'] / 100

In [ ]:
# Merge factor databases together, backfill rmw and cma for the 4 missing years :

df_ff = pd.merge(df_ff, df_ff5[['yyyymm','rmw','cma']], on=['yyyymm'], how='left', validate='many_to_one')

df_ff = pd.merge(df_ff, df_mom[['yyyymm','mom']], on=['yyyymm'], how='left', validate='many_to_one')

df_ff = df_ff.fillna(method='backfill')

print(df_ff.columns)

In [ ]:
# Merge onto df:

df = pd.merge(df, df_ff, on=['yyyymm'], how='left', validate='many_to_one')

df_ff = None 

## Macroeconomic predictors

In [ ]:
# Macroeconomic predictors from Amit Goyal’s website, retrieved in Nov. 2019

df_macropredictors = pd.read_csv('data/macropredictors.csv', sep=';')

df_macropredictors = df_macropredictors.drop(columns=['Index', 'csp'])

df_macropredictors.columns = map(str.lower, df_macropredictors.columns)

print(df_macropredictors.columns)

In [ ]:
# Merge onto df:

df = pd.merge(df, df_macropredictors, on=['yyyymm'], how='left', validate='many_to_one')

df_macropredictors = None

## Compustat Supplemental Short Interest File

In [ ]:
# Compustat Supplemental Short Interest File, retrieved Feb. 2020

df_shortinterest = pd.read_csv('data/shortinterest.csv')

df_shortinterest = df_shortinterest.sort_values(by=['gvkey', 'iid', 'datadate', 'splitadjdate'])
df_shortinterest = df_shortinterest.drop_duplicates(subset=['gvkey', 'iid', 'splitadjdate'], keep='last')

df_shortinterest = df_shortinterest[['gvkey', 'iid', 'splitadjdate', 'shortintadj']]
df_shortinterest = df_shortinterest.rename(columns={"splitadjdate": "date", "shortintadj": "shortint"})

#Shift observations from end of previous month to beginning of current month
df_shortinterest['shortint'] = df_shortinterest['shortint'].shift(periods=1)
df_shortinterest = df_shortinterest.dropna(subset=['shortint'])

print(df_shortinterest.columns)

## CRSP / Compustat Merged Security Monthly

In [ ]:
# CRSP / Compustat Merged Security Monthly, retrieved Feb. 2020 : drop Nan permnos, 
# only keep Common Shares Outstanding and Monthly Trading Volume

df_securitymonthly = pd.read_csv('data/securitymonthly.csv')

df_securitymonthly.columns = map(str.lower, df_securitymonthly.columns)
df_securitymonthly = df_securitymonthly.rename(columns={"datadate": "date", "lpermno": "permno"})

df_securitymonthly = df_securitymonthly.dropna(axis='rows', subset=['permno'])
df_securitymonthly = df_securitymonthly[['gvkey', 'iid', 'permno', 'date', 'cshtrm']]

In [ ]:
# Merge Short Interest File onto Security Monthly using gvkey, iid, date and sic: should be one-to-one, but sadly isnt...
# Collapse rows that are undiscernable in df by summing them first for each variable: dropping nas for each variable
# first allows us to keep missing observations while summing over groups of observations that have missing values and
# treating these as zeros; then compute short interest ratio (short interest over common shares outsanding)
# and trading volume ratio (monthly trading volume over common shares outstanding) -> It would be better to
# compute ratios first and then aggregate using weighed averages, but this is not possible with the available data...

df_securitymonthly = pd.merge(df_securitymonthly, df_shortinterest, on=['gvkey', 'iid', 'date'], how='left', validate='many_to_one')

df_securitymonthly_shortint = df_securitymonthly[['permno', 'date', 'shortint']]
df_securitymonthly_shortint = df_securitymonthly_shortint.dropna(axis='rows', subset=['shortint'])
df_securitymonthly_shortint = df_securitymonthly_shortint.groupby(['permno', 'date']).sum().reset_index()

df_securitymonthly_cshtrm = df_securitymonthly[['permno', 'date', 'cshtrm']]
df_securitymonthly_cshtrm = df_securitymonthly_cshtrm.dropna(axis='rows', subset=['cshtrm'])
df_securitymonthly_cshtrm = df_securitymonthly_cshtrm.groupby(['permno', 'date']).sum().reset_index()

df_securitymonthly = df_securitymonthly[['permno', 'date']]
df_securitymonthly = df_securitymonthly.drop_duplicates(subset=['permno', 'date'], keep='first')

df_securitymonthly = pd.merge(df_securitymonthly, df_securitymonthly_shortint, on=['permno', 'date'], how='left', validate='many_to_one')
df_securitymonthly = pd.merge(df_securitymonthly, df_securitymonthly_cshtrm, on=['permno', 'date'], how='left', validate='many_to_one')

print(df_securitymonthly.columns)

## Final dataframe merge, cleaning and sorting

In [ ]:
# Final merge onto df:

df = pd.merge(df, df_securitymonthly, on=['permno', 'date'], how='left', validate='many_to_one')

df_securitymonthly = None
df_shortinterest = None

In [ ]:
# Removing unneeded variables, deleting observations with missing date, permno or return

df = df.drop(columns=['yyyymm'])

df['ret'] = pd.to_numeric(df['ret'], errors='coerce')

df = df.dropna(axis='rows', subset=['date', 'permno', 'ret'])

In [ ]:
# Removing observations before 1958 (we do not have 1957 in full)

df = df[df['date']>=19580131]

In [ ]:
# Sorting columns alphabetically, sorting rows by date and permno

df = df[['date','permno','ret'] + sorted(list(set(df.columns) - set(['date','permno','ret'])))]

df = df.sort_values(by=['date','permno'])

df = df.reset_index(drop=True)

In [ ]:
dates = np.unique(df['date'].to_numpy())

dtdates = pd.to_datetime(dates, format='%Y%m%d')

permnos = np.unique(df['permno'].to_numpy())

vrs = np.setdiff1d(np.array(df.columns), ['date','permno'])

facs = np.setdiff1d(vrs, ['ret','shrout','cshtrm','shortint'])

T, N, V, F = len(dates), len(permnos), len(vrs), len(facs)

returns = df[['date','permno','ret',]].pivot(index='date', columns='permno', values='ret')

## Creating df_ml and df_mktvars

In [ ]:
df_ml = df.drop(columns=['shrout','cshtrm','shortint'])

df_ml = pd.DataFrame(scale(df_ml, axis=0), index=df_ml.index,columns=df_ml.columns)

df_ml[['date', 'permno', 'ret']] = df[['date', 'permno', 'ret']]

In [ ]:
df_mktvars = df[['date', 'permno', 'ret', 'mktcap', 'shrout','cshtrm','shortint']]

## Macro / market time series

In [ ]:
macrofacs = ['mkt_rf', 'smb', 'hml', 'rmw', 'cma', 'mom', 'rf', 'd12', 'e12',
             'bm', 'tbl', 'aaa', 'baa', 'lty', 'ntis', 'rfree', 'infl', 
             'ltr', 'corpr', 'svar', 'crsp_spvw', 'crsp_spvwx']

macrodf = df.loc[:,macrofacs + ['date']]
macrodf = macrodf.drop_duplicates(subset='date')
macrodf = macrodf.set_index('date')

rf = macrodf['rf']
mkt_rf = macrodf['mkt_rf']
smb = macrodf['smb']
hml = macrodf['hml']
rmw = macrodf['rmw']
cma = macrodf['cma']
mom = macrodf['mom']

## Saving datasets

In [ ]:
pickle.dump(df, open('data/df', 'wb'))

pickle.dump(df_ml, open('data/df_ml', 'wb'))

pickle.dump(df_mktvars, open('data/df_mktvars', 'wb'))

pickle.dump([dates, dtdates, permnos, vrs, facs, returns, T, N, V, F], open('data/info', 'wb'))

pickle.dump([rf, mkt_rf, smb, hml, rmw, cma, mom], open('data/misc', 'wb'))